# 1655 

In the movie "Blues Brothers", the orphanage where Elwood and Jake were raised may be sold to the Board of Education if they do not pay 5000 dollars in taxes at the Cook County Assessor's Office in Chicago. After playing a gig in the Palace Hotel ballroom to earn these 5000 dollars, they have to find a way to Chicago. However, this is not so easy as it sounds, since they are chased by the Police, a country band and a group of Nazis. Moreover, it is 106 miles to Chicago, it is dark and they are wearing sunglasses.

As they are on a mission from God, you should help them find the safest way to Chicago. In this problem, the safest way is considered to be the route which maximises the probability that they are not caught.

## Input

The input file contains several test cases.

Each test case starts with two integers n and m (2 ≤ n ≤ 100 , 1 ≤ m ≤ n*(n-1)/2). n is the number of intersections, m is the number of streets to be considered.

The next m lines contain the description of the streets. Each street is described by a line containing 3 integers a, b and p (1 ≤ a, b ≤ n , a ≠ b, 1 ≤ p ≤ 100): a and b are the two end points of the street and p is the probability in percent that the Blues Brothers will manage to use this street without being caught. Each street can be used in both directions. You may assume that there is at most one street between two end points.

The last test case is followed by a zero.

Obs.: The safest path for the sample input is 1 -> 4 -> 3 -> 5

- n = arestas
- m = vértices

- a = vertice source
- b = vertice destino
- p = peso

## Output

For each test case, calculate the probability of the safest path from intersection 1 (the Palace Hotel) to intersection n (the Honorable Richard J. Daley Plaza in Chicago). You can assume that there is at least one path between intersection 1 and n.

Print the probability as a percentage with exactly 6 digits after the decimal point. The percentage value is considered correct if it differs by at most 10-6 from the judge output. Adhere to the format shown below and print one line for each test case.

- safest path from 1 to n
- print as a percentage of 6 digits after ,. 

## Developtment

In [1]:
"""Same as before however added method for listing edges"""


class Node:  
    def __init__(self, name, color=None, d=None, pi=None):
        self.name = name
        self.adj_list = []
        self.color = color  # estado: visitado, não visitado, explorado
        self.d = d  # distancia do vértice origem
        self.pi = pi  # vértice predecessor
   
    def __repr__(self):
        return repr(self.name)
    
class Graph:
    def __init__(self, num_vertices=None, undirected=True):
        self.num_vertices = num_vertices
        self.nodes = [Node(v) for v in range(num_vertices)]
        self.undirected = undirected

    def __getitem__(self, item):
        return self.nodes[item]
    
    def add_edge(self, source, dest, weight):
        self.nodes[source].adj_list.append((self.nodes[dest], weight))
        if self.undirected:
            self.nodes[dest].adj_list.append((self.nodes[source], weight))
        
    def print(self):
        for node in self.nodes:
            print(f"Node {node.name} -> {[(n.name, w) for n, w in node.adj_list]}")
            
    def get_list_edges(self):
        edges = []
        for u in self.nodes:
            for v, w in u.adj_list:
                if self.undirected:  # So it won't repeat edges
                    first = min(u.name, v.name)
                    second = v.name if first == u.name else u.name
                    edge = (first, second, w)
                    edges.append(edge) if edge not in edges else None
                else:
                    edges.append((u.name, v.name, w))

        return edges
    
    def get_sorted_list_edges(self):
        edges = self.get_list_edges()
        return sorted(edges, key=lambda tup: tup[2])

    def find_edge_weight(self, e1, e2):
        for v, w in self.nodes[e1].adj_list:
            if v.name == e2:
                return w
        return 'Not found'

        

In [2]:
class PriorityQueue(object):
    def __init__(self):
        self.queue = []
  
    def __str__(self):
        return ' '.join([str(i) for i in self.queue])
  
    # for checking if the queue is empty
    def isEmpty(self):
        return len(self.queue) == 0
  
    # for inserting an element in the queue
    def insert(self, data):
        self.queue.append(data)
  
    # for popping an element based on Priority
    def extract_max(self):
        try:
            max_val = 0
            for i in range(len(self.queue)):
                if self.queue[i].d > self.queue[max_val].d:
                    max_val = i
            item = self.queue[max_val]
            del self.queue[max_val]
            return item
        except IndexError:
            print()
            exit()
            
    def extract_min(self):
        try:
            min_val = 0
            for i in range(len(self.queue)):
                if self.queue[i].d < self.queue[min_val].d:
                    min_val = i
            item = self.queue[min_val]
            del self.queue[min_val]
            return item
        except IndexError:
            print()
            exit()
    
    def decrease_key(self, v, d):
        for i, n in enumerate(self.queue):
            if n.name == v.name:
                n.d = d           

In [3]:
with open("input", "r") as f:
    inpts = f.read()
    inpts = inpts.split('\n')

inpts

['5 7',
 '5 2 100',
 '3 5 80',
 '2 3 70',
 '2 1 50',
 '3 4 90',
 '4 1 85',
 '3 1 70',
 '5 8',
 '5 2 100',
 '3 5 80',
 '2 3 70',
 '2 1 50',
 '3 4 90',
 '4 1 85',
 '3 1 70',
 '4 5 100',
 '0',
 '']

In [324]:
# Read input
counter = 0
for inpt in inpts:
    counter += 1
    if counter == 1:
        n = int(inpt[0])
        m = int(inpt[2])
        g = Graph(n+1)
    elif inpt == '0':
        break
    else:
        a = int(inpt[0])
        b = int(inpt[2])
        p = int(inpt[4:])/100
        g.add_edge(a, b, p)
del counter, a, b, p, m

In [4]:
INF = 1

class Dijkstra():
    def __init__(self, G):
        self.G = G
        self.E = G.get_list_edges()
        
    def initialize_single_source(self, s):
        """Inicialização dos nós com distancias infinitas até a 
        source e predecessores nulos"""
        
        for v in self.G.nodes:
            v.d = 1
            v.pi = None
            v.color = 'white'
            
        self.G[s].d = 1
        self.G[s].color = 'gray'

    def relax(self, u, v, w):
        """Verifica se o o caminho para o vertice `u` melhora 
        o caminho minimo encontrado até agora para o vértice `v`.
        Se sim, atualiza o valor de `v.d` com o peso até agora, 
        e adiciona `u` como seu predecessor."""
        
        increased = False
        calculation = self.G[u].d * w
        
        print(f"new distance: {calculation}")
        
        if self.G[v].color == 'white' and self.G[v].d==1:
            increased = True if calculation > self.G[v].d else False
            self.G[v].d = calculation
            self.G[v].pi = self.G[u]
            print(f"updating {v} distance with {calculation}")
        
        # [print(f"Vertex: {n.name}, shortest_distance_from_source: {n.pi}") for n in self.G.nodes]
        
        if self.G[v].d < calculation:
            increased = True if calculation > self.G[v].d else False
            self.G[v].d = calculation
            self.G[v].pi = self.G[u]
            print(f"updating {v} distance with {calculation}")
    
        self.G[v].color = 'gray'
        
            
        return increased
        
    def dijkstra(self, s, chicago):
        self.initialize_single_source(s)
        
        S = set()
        Q = PriorityQueue()
        Q.insert(self.G.nodes[s])
            
        while Q.queue != []:
            u = Q.extract_max()
            S = S.union({u})
            for v, w in u.adj_list:
                print(f"checking: {u}-{v}")
                if v.color != 'black':
                    
                    increased = self.relax(u.name, v.name, w)
                    if increased:
                        Q.decrease_key(v, v.d)
                    Q.insert(v)
                else:
                    print(f"Jumping {u}-{v} because {v} is black")
            u.color = 'black'
            print(f"paiting {u} black")
            print()
            
        
        [print(f"Vertex: {n.name}, highest probability of not being caught: {n.d}") for n in S]
        for n in S:
            if n.name == chicago:
                print(f"From 1 to chicago: {n.d}")
                break

In [392]:
g.print()

Node 0 -> []
Node 1 -> [(2, 0.5), (4, 0.85), (3, 0.7)]
Node 2 -> [(5, 1.0), (3, 0.7), (1, 0.5)]
Node 3 -> [(5, 0.8), (2, 0.7), (4, 0.9), (1, 0.7)]
Node 4 -> [(3, 0.9), (1, 0.85)]
Node 5 -> [(2, 1.0), (3, 0.8)]


In [393]:
d = Dijkstra(g)
d.dijkstra(1, 5)

checking: 1-2
new distance: 0.5
updating 2 distance with 0.5
checking: 1-4
new distance: 0.85
updating 4 distance with 0.85
checking: 1-3
new distance: 0.7
updating 3 distance with 0.7
paiting 1 black

checking: 4-3
new distance: 0.765
updating 3 distance with 0.765
checking: 4-1
Jumping 4-1 because 1 is black
paiting 4 black

checking: 3-5
new distance: 0.6120000000000001
updating 5 distance with 0.6120000000000001
checking: 3-2
new distance: 0.5355
updating 2 distance with 0.5355
checking: 3-4
Jumping 3-4 because 4 is black
checking: 3-1
Jumping 3-1 because 1 is black
paiting 3 black

checking: 3-5
new distance: 0.6120000000000001
checking: 3-2
new distance: 0.5355
checking: 3-4
Jumping 3-4 because 4 is black
checking: 3-1
Jumping 3-1 because 1 is black
paiting 3 black

checking: 5-2
new distance: 0.6120000000000001
updating 2 distance with 0.6120000000000001
checking: 5-3
Jumping 5-3 because 3 is black
paiting 5 black

checking: 2-5
Jumping 2-5 because 5 is black
checking: 2-3
Jumpi

## Adjust

In [9]:
class Node:  
    def __init__(self, name, color=None, d=None, pi=None):
        self.name = name
        self.adj_list = []
        self.color = color  # estado: visitado, não visitado, explorado
        self.d = d  # distancia do vértice origem
        self.pi = pi  # vértice predecessor
   
    def __repr__(self):
        return repr(self.name)
    
class Graph:
    def __init__(self, num_vertices=None, undirected=True):
        self.num_vertices = num_vertices
        self.nodes = [Node(v) for v in range(num_vertices)]
        self.undirected = undirected

    def __getitem__(self, item):
        return self.nodes[item]
    
    def add_edge(self, source, dest, weight):
        self.nodes[source].adj_list.append((self.nodes[dest], weight))
        if self.undirected:
            self.nodes[dest].adj_list.append((self.nodes[source], weight))
        
    def print(self):
        for node in self.nodes:
            print(f"Node {node.name} -> {[(n.name, w) for n, w in node.adj_list]}")
            
    def get_list_edges(self):
        edges = []
        for u in self.nodes:
            for v, w in u.adj_list:
                if self.undirected:  # So it won't repeat edges
                    first = min(u.name, v.name)
                    second = v.name if first == u.name else u.name
                    edge = (first, second, w)
                    edges.append(edge) if edge not in edges else None
                else:
                    edges.append((u.name, v.name, w))

        return edges
    
    def get_sorted_list_edges(self):
        edges = self.get_list_edges()
        return sorted(edges, key=lambda tup: tup[2])

    def find_edge_weight(self, e1, e2):
        for v, w in self.nodes[e1].adj_list:
            if v.name == e2:
                return w
        return 'Not found'

        
        
class PriorityQueue(object):
    def __init__(self):
        self.queue = []
  
    def __str__(self):
        return ' '.join([str(i) for i in self.queue])
  
    # for checking if the queue is empty
    def isEmpty(self):
        return len(self.queue) == 0
  
    # for inserting an element in the queue
    def insert(self, data):
        self.queue.append(data)
  
    # for popping an element based on Priority
    def extract_max(self):
        try:
            max_val = -999999999
            for i in range(len(self.queue)):
                if self.queue[i].d > self.queue[max_val].d:
                    max_val = i
            item = self.queue[max_val]
            del self.queue[max_val]
            return item
        except IndexError:
            print()
            exit()
    
    def decrease_key(self, v, d):
        for i, n in enumerate(self.queue):
            if n.name == v.name:
                n.d = d           


class Dijkstra():
    def __init__(self, G):
        self.G = G
        
    def initialize_single_source(self, s):
        """Inicialização dos nós com distancias infinitas até a 
        source e predecessores nulos"""
        
        for v in self.G.nodes:
            v.d = 1
            v.pi = None
            v.color = 'white'
            
        self.G[s].d = 1
        self.G[s].color = 'gray'

    def relax(self, u, v, w):
        """Verifica se o o caminho para o vertice `u` melhora 
        o caminho minimo encontrado até agora para o vértice `v`.
        Se sim, atualiza o valor de `v.d` com o peso até agora, 
        e adiciona `u` como seu predecessor."""
        
        increased = False
        calculation = self.G[u].d * w
        
        if self.G[v].color == 'white' and self.G[v].d==1:
            increased = True if calculation > self.G[v].d else False
            self.G[v].d = calculation
            self.G[v].pi = self.G[u]
                
        if self.G[v].d < calculation:
            increased = True if calculation > self.G[v].d else False
            self.G[v].d = calculation
            self.G[v].pi = self.G[u]
    
        self.G[v].color = 'gray'
        
        return increased
        
    def dijkstra(self, s, chicago):
        self.initialize_single_source(s)
        
        S = set()
        Q = PriorityQueue()
        Q.insert(self.G.nodes[s])
            
        while Q.queue != []:
            u = Q.extract_max()
            for v, w in u.adj_list:
                if v.color != 'black':
                    increased = self.relax(u.name, v.name, w)
                    if increased:
                        Q.decrease_key(v, v.d)
                    Q.insert(v)
            u.color = 'black'
        
        print("{:.6f} percent".format(self.G.nodes[chicago].d*100))

with open("input2", "r") as f:
    inpts = f.read()
    inpts = inpts.split('\n')

counter = 0
for inpt in inpts:
# while True:
    # inpt = input()
    inpt = inpt.split(' ')
    if len(inpt) == 2:
        n = int(inpt[0])
        m = int(inpt[1])
        g = Graph(n+1)
    elif len(inpt) == 1:
        break
    elif len(inpt) > 2 and counter < m:
        counter += 1
        a = int(inpt[0])
        b = int(inpt[1])
        p = int(inpt[2])/100
        g.add_edge(a, b, p)

    if counter == m:
        counter = 0
        d = Dijkstra(g)
        d.dijkstra(1, n)
        

AttributeError: 'NoneType' object has no attribute 'adj_list'

## Trying to optmize

In [453]:

# Read input
counter = 0
g = []
for inpt in inpts:
    counter += 1
    if counter == 1:
        vertices = int(inpt[0])
        edges = int(inpt[2])
        g = g + [{} for i in range(vertices)]
    elif inpt == '0':
        break
    else:
        a = int(inpt[0])-1
        b = int(inpt[2])-1
        p = int(inpt[4:])/100

        g[a][b] = (p, 1)
        g[b][a] = (p, 1)

del counter, a, b, p, edges

for i, val in enumerate(g):
    print(f"{i}: {val}")

s = 0
Q = PriorityQueue()
for v in g.keys():
    Q.insert(v)

while Q.queue != []:
    for adj in g[s].keys():
        print(adj)
        
    current = Q.extract_max()
    for adj, w in current.keys():
        
        increased = False
        calculation = adj[55] * w
        print(calculation,self.G[v].d, self.G[u].name)

        # [print(f"Vertex: {n.name}, shortest_distance_from_source: {n.pi}") for n in self.G.nodes]
        if self.G[v].d < calculation:
            increased = True if calculation > self.G[v].d else False
            self.G[v].d = calculation
            self.G[v].pi = self.G[u]
            
        if decreased:
            Q.decrease_key(v, v.d)
            
    break

ValueError: invalid literal for int() with base 10: ''